In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import *

catalog_name='olist'

### Load Bronze Tables

In [0]:
orders_brz = spark.table(f"{catalog_name}.bronze.brz_orders")
customers_brz = spark.table(f"{catalog_name}.bronze.brz_customers")
order_items_brz = spark.table(f"{catalog_name}.bronze.brz_order_items")
payments_brz = spark.table(f"{catalog_name}.bronze.brz_payments")
reviews_brz = spark.table(f"{catalog_name}.bronze.brz_reviews")
products_brz = spark.table(f"{catalog_name}.bronze.brz_products")
sellers_brz = spark.table(f"{catalog_name}.bronze.brz_sellers")
geo_brz = spark.table(f"{catalog_name}.bronze.brz_geolocation")
category_map_brz = spark.table(
    f"{catalog_name}.bronze.brz_product_category_translation"
)

In [0]:
meta_cols = ["_ingested_at", "_source_file"]

### Clean Orders

In [0]:

# Remove duplicates and ensure valid timestamps
orders_slv = (
    orders_brz
    .dropDuplicates(["order_id"])
    .drop(*meta_cols)
    .withColumn(
        "order_purchase_date",
        F.to_date("order_purchase_timestamp")
    )
)

### Clean Payments

In [0]:
payments_slv = (
    payments_brz
    .drop(*meta_cols)
    .groupBy("order_id")
    .agg(
        F.sum("payment_value").alias("total_payment_value"),
        F.avg("payment_installments").alias("avg_installments")
    )
)

### Clean Customers

In [0]:
customers_slv = customers_brz.drop(*meta_cols)

### Clean Reviews

In [0]:
reviews_slv = (
    reviews_brz
    .drop(*meta_cols)
    .groupBy("order_id")
    .agg(
        F.avg("review_score").alias("avg_review_score")
    )
)

### Clean Products & Category Translation

In [0]:
products_slv = (
    products_brz
    .drop(*meta_cols)
    .join(
        category_map_brz,
        "product_category_name",
        "left"
    )
)

In [0]:
# Joining order_items and products dataset
order_items_slv = (
    order_items_brz
    .drop(*meta_cols)
    .join(products_slv, "product_id", "left")
)

### Core Fact Table

In [0]:
order_fact_slv = (
    orders_slv
    .join(customers_slv, "customer_id")
    .join(order_items_slv, "order_id")
    .join(payments_slv, "order_id")
    .join(reviews_slv, "order_id", "left")
)

### Delivery Metrics

In [0]:
order_fact_slv = (
    order_fact_slv
    .withColumn(
        "delivery_days",
        F.datediff(
            F.col("order_delivered_customer_date"),
            F.col("order_purchase_timestamp")
        )
    )
    .withColumn(
        "delivery_delay_days",
        F.datediff(
            F.col("order_delivered_customer_date"),
            F.col("order_estimated_delivery_date")
        )
    )
)

### Save Silver Fact Table

In [0]:
order_fact_slv.write.format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .saveAsTable(f"{catalog_name}.silver.slv_order_fact")

### Save Silver Dimension Tables

In [0]:
customers_slv.write.format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .saveAsTable(f"{catalog_name}.silver.slv_customers")

products_slv.write.format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .saveAsTable(f"{catalog_name}.silver.slv_products")